In [1]:
import sys
import os

# Add the upstream directory to sys.path
upstream_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if upstream_dir not in sys.path:
    sys.path.insert(0, upstream_dir)

# Now you can import the module
from opentrons_api import ot2_api
from microtissue_manipulator import core, utils
import numpy as np 
import cv2
import time
import json
import keyboard
# from pynput import keyboard
from configs import paths
import matplotlib.pyplot as plt
import requests
import datetime

In [2]:
cap = core.Camera(0, config_profile='standardDeck', use_new_cam_mtx=True)

Using camera without buffer ...
Camera initialized ...


# Video test

In [3]:
window = cap.get_window()

robot_coords = []
camera_coords = []

while True:
    frame = cap.get_frame(undist=True)
    cv2.imshow(cap.window_name, frame)

    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        break

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(os.path.join(paths.BASE_DIR, 'outputs', 'images', 'segmentation_test', filename), frame)
        print(f"Frame saved as {filename}")
   
cv2.destroyAllWindows()

# Init

In [3]:
openapi = ot2_api.OpentronsAPI()

In [5]:
openapi.toggle_lights()

<Response [200]>

In [6]:
openapi.home_robot()

Request status:
<Response [200]>
{
  "message": "Homing robot."
}


<Response [200]>

In [6]:
r = openapi.get_run_info()

Total number of runs: 20
Current run ID: 76358a1e-dcbd-445c-b4ab-9255c79da526
Current run status: idle


In [7]:
openapi.create_run(verbose=True)

Request status:
<Response [201]>
{
  "data": {
    "id": "76358a1e-dcbd-445c-b4ab-9255c79da526",
    "ok": true,
    "createdAt": "2025-01-26T10:06:28.552400+00:00",
    "status": "idle",
    "current": true,
    "actions": [],
    "errors": [],
    "hasEverEnteredErrorRecovery": false,
    "pipettes": [],
    "modules": [],
    "labware": [],
    "liquids": [],
    "labwareOffsets": [],
    "runTimeParameters": [],
    "outputFileIds": []
  }
}


<Response [201]>

In [8]:
openapi.load_pipette()

Request status:
<Response [201]>
{
  "data": {
    "id": "c1ab637f-6c11-468a-8c14-0375ef1f9c67",
    "createdAt": "2025-01-26T10:06:53.876961+00:00",
    "commandType": "loadPipette",
    "key": "c1ab637f-6c11-468a-8c14-0375ef1f9c67",
    "status": "succeeded",
    "params": {
      "pipetteName": "p300_single_gen2",
      "mount": "left"
    },
    "result": {
      "pipetteId": "6a51cb8a-aa10-45a7-bb93-666fb4664a70"
    },
    "startedAt": "2025-01-26T10:06:53.879425+00:00",
    "completedAt": "2025-01-26T10:06:55.930480+00:00",
    "intent": "setup",
    "notes": []
  }
}


<Response [201]>

# Labware declaration

### Opentrons 300 ul

In [10]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "opentrons_96_tiprack_300ul"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 11)

Labware ID:
ed963615-bb7a-47ab-9cbc-7bb8ef661681



<Response [201]>

### VWR 200 ul XL

In [9]:
custom_labware_path = os.path.join(paths.BASE_DIR,'protocols','vwr_96_tiprack_200ul_xl.json')
with open(custom_labware_path, 'r') as json_file:
    custom_labware = json.load(json_file)

command_dict = {
            "data": custom_labware
        }

command_payload = json.dumps(command_dict)

url = openapi.get_url('runs')+ f'/{openapi.run_id}/'+ 'labware_definitions'
r = requests.post(url = url, headers = openapi.HEADERS, params = {"waitUntilComplete": True}, data = command_payload)

In [11]:
#Define a tip rack. This is the default tip rack for the robot.
TIP_RACK = "vwr_96_tiprack_200ul_xl"
#Load the tip rack. Slot = 1 by default.
openapi.load_labware(TIP_RACK, 10, namespace='custom_beta',verbose=True)

Labware ID:
8f3a0bf5-2958-4ac5-abf0-7e213fdc1c73



<Response [201]>

In [12]:
RESERVOIR = "corning_6_wellplate_16.8ml_flat"
openapi.load_labware(RESERVOIR, 3, namespace='opentrons',verbose=True)

Labware ID:
492ea99c-5b92-4999-816b-6cda6d1c1d6f



<Response [201]>

In [13]:
# WELL_PLATE = "corning_96_wellplate_360ul_flat"
WELL_PLATE = "corning_384_wellplate_112ul_flat"
openapi.load_labware(WELL_PLATE, 6, namespace='opentrons',verbose=True)

Labware ID:
864080eb-fa25-478c-89a5-6e026a3b5294



<Response [201]>

In [14]:
r = openapi.pick_up_tip(openapi.labware_dct['10'], "A1")

In [14]:
json.loads(r.text)

{'data': {'id': '2e9895b7-7b57-4a12-9fea-67e3c1d44c91',
  'createdAt': '2025-01-25T02:09:05.698416+00:00',
  'commandType': 'pickUpTip',
  'key': '2e9895b7-7b57-4a12-9fea-67e3c1d44c91',
  'status': 'failed',
  'params': {'pipetteId': 'd8c403d6-c4fb-471b-8545-5cc147fde110',
   'labwareId': '38304447-0c57-4244-a527-956f37e2f285',
   'wellName': 'A2',
   'wellLocation': {'origin': 'top',
    'offset': {'x': 0.0, 'y': 0.0, 'z': 0.0}}},
  'error': {'id': 'afe5b47c-2d6c-46f4-b7c8-c46b1662e278',
   'createdAt': '2025-01-25T02:09:07.528322+00:00',
   'isDefined': False,
   'errorType': 'UnexpectedTipAttachError',
   'errorCode': '3012',
   'detail': 'Cannot perform pick_up_tip with a tip already attached.',
   'errorInfo': {},
   'wrappedErrors': []},
  'startedAt': '2025-01-25T02:09:05.700520+00:00',
  'completedAt': '2025-01-25T02:09:07.528322+00:00',
  'intent': 'setup',
  'notes': [{'noteKind': 'debugErrorRecovery',
    'shortMessage': 'Handling this command failure with FAIL_RUN.',
    'l

In [24]:
r = openapi.pick_up_tip(openapi.labware_dct['11'], "A1")

In [23]:
openapi.drop_tip_in_place()

<Response [201]>

In [15]:
openapi.drop_tip(openapi.labware_dct['10'], "A1")

<Response [201]>

In [64]:
openapi.aspirate(openapi.labware_dct['9'], "A1", well_location='center', volume = 200, flow_rate = 200)

<Response [201]>

# Filling a well plate

In [17]:
openapi.drop_tip_in_place()

<Response [201]>

In [22]:
# Take solution off the well palte with cuboids

columns = list(range(1,5))
rows = ['I']

row_idx = 0
column_idx = 0

while row_idx < len(rows):
    row = rows[row_idx]
    while column_idx < len(columns):
        column = columns[column_idx]
        r = openapi.aspirate(openapi.labware_dct['6'], f"{row}{column}", well_location = 'bottom', offset = (0,0,0), volume = 50, flow_rate = 10)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
                print('Dumping fluid')
                openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
        else:
            column_idx += 1
    column_idx = 0
    row_idx += 1

openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.move_relative('z', 20)


<Response [201]>

In [17]:
json.loads(r.text)

{'data': {'id': 'be730250-eb5e-4914-98fe-068c40074a03',
  'createdAt': '2025-01-26T13:23:17.953899+00:00',
  'commandType': 'aspirate',
  'key': 'be730250-eb5e-4914-98fe-068c40074a03',
  'status': 'failed',
  'params': {'labwareId': '864080eb-fa25-478c-89a5-6e026a3b5294',
   'wellName': 'F9',
   'wellLocation': {'origin': 'bottom',
    'offset': {'x': 0.0, 'y': 0.0, 'z': -0.5},
    'volumeOffset': 0.0},
   'flowRate': 5.0,
   'volume': 50.0,
   'pipetteId': '6a51cb8a-aa10-45a7-bb93-666fb4664a70'},
  'error': {'id': '52123ebd-5d9c-4c7a-8b96-0b30a94d2201',
   'createdAt': '2025-01-26T13:23:19.772940+00:00',
   'isDefined': False,
   'errorType': 'OperationLocationNotInWellError',
   'errorCode': '4000',
   'detail': 'Specifying bottom with an offset of x=0.0 y=0.0 z=-0.5 and a volume offset of 0.0 results in an operation location below the bottom of the well',
   'errorInfo': {},
   'wrappedErrors': []},
  'startedAt': '2025-01-26T13:23:17.955931+00:00',
  'completedAt': '2025-01-26T13:2

In [136]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)

<Response [201]>

In [130]:
r = openapi.aspirate(openapi.labware_dct['6'], f"A5", well_location = 'bottom', volume = 70, flow_rate = 5)

In [137]:
openapi.move_relative('z', 20)

<Response [201]>

In [15]:
columns = list(range(1,25))
rows = ['F', 'G', 'H', 'I']

row_idx = 0
column_idx = 0

while row_idx < len(rows):
    row = rows[row_idx]
    while column_idx < len(columns):
        column = columns[column_idx]
        r = openapi.dispense(openapi.labware_dct['6'], f"{row}{column}", well_location = 'bottom', volume = 50, flow_rate = 200)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidDispenseVolumeError':
                print('Refilling pipette')
                openapi.aspirate(openapi.labware_dct['3'], "A1", well_location ='bottom', volume = 200, flow_rate = 200)
        else:
            column_idx += 1
    column_idx = 0
    row_idx += 1

openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)
openapi.dispense(openapi.labware_dct['3'], "A2", volume = 50, flow_rate = 200)

Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette
Refilling pipette


<Response [201]>

In [ ]:
json.loads(r.text)['data']['error']['errorType']

{'id': 'dec84017-3246-49d5-8548-ccfe85aa2350',
 'createdAt': '2024-11-02T03:44:26.715454+00:00',
 'isDefined': False,
 'errorType': 'InvalidDispenseVolumeError',
 'errorCode': '4000',
 'detail': 'Cannot dispense 50.0 µL when only 0.0 µL has been aspirated.',
 'errorInfo': {},
 'wrappedErrors': []}

# Robot <-> camera calibration

In [9]:
squaresX=7
squaresY=5 
squareLength=0.022
markerLength=0.011
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, params)
board = cv2.aruco.CharucoBoard((squaresX, squaresY), squareLength, markerLength, aruco_dict)

manual_movement = utils.ManualRobotMovement(openapi)

openapi.move_to_coordinates((155,47,100), min_z_height=1, verbose=False)

window = cap.get_window()

robot_coords = []
camera_coords = []

while True:
 # Capture frame-by-frame
    frame = cap.get_frame(undist=True)

    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 100), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Calculate the center of each quarter of the screen
    quarter_centers = [
        (center_screen_x // 2, center_screen_y // 2),
        (3 * center_screen_x // 2, center_screen_y // 2),
        (center_screen_x // 2, 3 * center_screen_y // 2),
        (3 * center_screen_x // 2, 3 * center_screen_y // 2)
    ]

    # Draw circles at the center of each quarter
    for qx, qy in quarter_centers:
        cv2.circle(frame, (qx, qy), 5, (0, 255, 255), -1)
        cv2.putText(frame, f"({qx}, {qy})", (qx + 10, qy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    if marker_corners:
        for corner in marker_corners:
            corner = corner.reshape((4, 2))
            for point in corner:
                cv2.circle(frame, tuple(point.astype(int)), 5, (0, 255, 0), -1)

            center_x = int(np.mean(corner[:, 0]))
            center_y = int(np.mean(corner[:, 1]))
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            cv2.putText(frame, f"({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Calculate side lengths
    side_lengths = []
    if marker_corners:
        for corner in marker_corners[0]:
            for i in range(4):
                side_length = np.linalg.norm(corner[i] - corner[(i + 1) % 4])
                side_lengths.append(side_length)

    # Calculate the average side length
        average_side_length = np.mean(side_lengths)
        area = cv2.contourArea(marker_corners[0])
        one_d_ratio = 13.83 / average_side_length
        size_conversion_ratio = 13.83 ** 2 / area
        cv2.putText(frame, f"Area of marker: {area:.2f}", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow(cap.window_name, frame)

    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break
    elif key_pressed == ord('s'):
        x, y, z = openapi.get_position(verbose=False)[0].values()
        robot_coords.append((x, y))
        camera_coords.append((center_x, center_y))

# When everything done, release the capture
# cap.release_camera()
cv2.destroyAllWindows()

calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)
calibration_data['size_conversion_ratio'] = size_conversion_ratio
calibration_data['one_d_ratio'] = one_d_ratio
utils.save_calibration_config(calibration_profile, calibration_data)

In [10]:
calibration_points = [(160.0, 42.0),
 (150.0, 42.0),
 (150.0, 52.00000000000001),
 (160.0, 52.00000000000001)]

robot_coords = []
camera_coords = []


window = cap.get_window()

for calib_pt in calibration_points:
    openapi.move_to_coordinates((*calib_pt, 100), min_z_height=1, verbose=False)
    cv2.waitKey(1000)
    frame = cap.get_frame(undist=True)
    
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Calculate the center of each quarter of the screen
    quarter_centers = [
        (center_screen_x // 2, center_screen_y // 2),
        (3 * center_screen_x // 2, center_screen_y // 2),
        (center_screen_x // 2, 3 * center_screen_y // 2),
        (3 * center_screen_x // 2, 3 * center_screen_y // 2)
    ]

    # Draw circles at the center of each quarter
    for qx, qy in quarter_centers:
        cv2.circle(frame, (qx, qy), 5, (0, 255, 255), -1)
        cv2.putText(frame, f"({qx}, {qy})", (qx + 10, qy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    marker_corners, marker_ids, _ = detector.detectMarkers(frame)
    if marker_corners:
        for corner in marker_corners:
            corner = corner.reshape((4, 2))
            for point in corner:
                cv2.circle(frame, tuple(point.astype(int)), 5, (0, 255, 0), -1)

            center_x = int(np.mean(corner[:, 0]))
            center_y = int(np.mean(corner[:, 1]))
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            cv2.putText(frame, f"({center_x}, {center_y})", (center_x + 10, center_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.waitKey(1)
    cv2.imshow(cap.window_name, frame)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    robot_coords.append((x, y))
    camera_coords.append((center_x, center_y))

cv2.destroyAllWindows()

In [87]:
openapi.blow_out_in_place(50)

<Response [201]>

In [88]:
openapi.aspirate(openapi.labware_dct['9'], "A1", well_location='center', volume = 100, flow_rate = 200)

<Response [201]>

In [89]:
openapi.dispense(openapi.labware_dct['9'], "A1", volume = 100, flow_rate = 200)

<Response [201]>

# Write transformation matrix

In [11]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

camera_coords = utils.sort_coordinates(camera_coords)
robot_coords = utils.sort_coordinates(robot_coords, reverse_y=True)

robot_to_camera_coords = {tuple(robot_coord): tuple(camera_coord) for robot_coord, camera_coord in zip(robot_coords, camera_coords)}
tf_mtx = utils.compute_tf_mtx(robot_to_camera_coords)

calibration_data['tf_mtx'] = tf_mtx.tolist()

utils.save_calibration_config(calibration_profile, calibration_data)

# Pipette offset calibration

In [13]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])


window = cap.get_window()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    if event == cv2.EVENT_LBUTTONDBLCLK:
        for contour in filtered_contours:
            r=cv2.pointPolygonTest(contour, (x,y), False)
            if r>0:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

                    x, y, _ = openapi.get_position(verbose=False)[0].values()
                    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                    X = X_init + diff[0] + offset[0]
                    Y = Y_init + diff[1] + offset[1]
                    
                    print(f"Robot coords: ({x}, {y})")
                    print(f"Clicked on: ({X}, {Y})")
                    openapi.move_to_coordinates((X, Y, 15), min_z_height=1, verbose=False)
                    # openapi.aspirate_in_place(flow_rate = 75, volume = 10)

                    
                else:
                    print("Contour center could not be found")

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 300
manual_movement = utils.ManualRobotMovement(openapi)

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.circle(frame, circle_center, circle_radius, (255, 0, 0), 2)
    # Create a mask with the same dimensions as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Draw a filled circle on the mask
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convert the masked frame to grayscale
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to the grayscale image
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    # Fill the area outside the circle with black pixels
    # Convert the mask to grayscale
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)


    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filter the contours to exclude the outermost
    # filtered_contours = [contour for contour, h in zip(contours, hei[0]) if h[3] == 1]
    # Filter the contours by size
    filtered_contours = [contour for contour in contours if 15 < cv2.contourArea(contour) < 500]
    # Draw the contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)




    cv2.imshow(cap.window_name, frame)
    key_pressed = cv2.waitKey(1)
    if key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X_init, y-Y_init]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = 75, volume = 10)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = 75, volume = 10)


    elif key_pressed == ord('q'):
        keyboard.unhook_all()
        break

        
cv2.destroyAllWindows()

Robot coords: (155.0, 47.0)
Clicked on: (203.75904964603245, 125.49591758314213)
Robot coords: (155.0, 47.0)
Clicked on: (220.66247034781193, 124.95123552626927)
Robot coords: (155.0, 47.0)
Clicked on: (203.4744896053945, 142.1715272775757)
Robot coords: (155.0, 47.0)
Clicked on: (195.08144072294667, 125.01653999162852)
Robot coords: (155.0, 47.0)
Clicked on: (198.27260121332654, 110.75007469055433)


# Move to point

In [35]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

def on_mouse_click(event, cX, cY, flags, param):
    global X_init, Y_init, X, Y, target_x, target_y
    if event == cv2.EVENT_LBUTTONDOWN:
        target_x, target_y = cX, cY
        # print("Clicked at pixel coordinate: ({}, {})".format(cX, cY))
        X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False)[0].values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X = X_init + diff[0] + offset[0]
        Y = Y_init + diff[1] + offset[1]
        # print(f"Robot coords: ({x}, {y})")
        # print(f"Clicked on: ({X}, {Y})")
        # openapi.move_to_coordinates((X, Y, 15), min_z_height=1)

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)



# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)

target_x, target_y = 0, 0

dish_bottom = 11.70# - 11.5
pickup_offset = 0.4 #0.6
flow_rate = 300
volume = 50

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 190), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Flow rate: {flow_rate} uL/s", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Volume: {volume} uL", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Pickup height: {dish_bottom + pickup_offset} mm", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    # cv2.circle(frame, (center_screen_x, center_screen_y), 5, (0, 0, 255), -1)
    cv2.circle(frame, (target_x, target_y), 3, (0, 0, 255), -1)
    # cv2.putText(frame, f"Center: ({center_screen_x}, {center_screen_y})", (center_screen_x + 10, center_screen_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    cv2.imshow(cap.window_name, frame)
    # Draw a point at the center of the screen
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

    elif key_pressed == ord('o'):
        x, y, _ = openapi.get_position(verbose=False)[0].values()

        calibration_data['offset'] = [x-X_init, y-Y_init]

        utils.save_calibration_config(calibration_profile, calibration_data)

    elif key_pressed == ord('d'):
        openapi.dispense_in_place(flow_rate = flow_rate, volume = volume)

    elif key_pressed == ord('b'):
        openapi.blow_out_in_place(50)

    elif key_pressed == ord('a'):
        openapi.aspirate_in_place(flow_rate = flow_rate, volume = volume, verbose=True)

    elif key_pressed == ord('p'):
        openapi.move_to_coordinates((X, Y, dish_bottom+pickup_offset), min_z_height=dish_bottom)

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+filename, frame)
        print(f"Frame saved as {filename}")


cv2.destroyAllWindows()

Frame saved as frame_2025_02_14_20-41-14.png
Request status:
<Response [201]>
{
  "data": {
    "id": "75aa88db-6ad3-439e-b1d8-1e3544b38680",
    "createdAt": "2025-01-26T09:54:12.642504+00:00",
    "commandType": "moveToCoordinates",
    "key": "75aa88db-6ad3-439e-b1d8-1e3544b38680",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 11.7,
      "forceDirect": false,
      "pipetteId": "3165cef1-4de0-400c-b3ab-020f0677a697",
      "coordinates": {
        "x": 203.89470887491026,
        "y": 123.2790046932126,
        "z": 12.1
      }
    },
    "result": {
      "position": {
        "x": 203.89470887491026,
        "y": 123.2790046932126,
        "z": 12.1
      }
    },
    "startedAt": "2025-01-26T09:54:12.644715+00:00",
    "completedAt": "2025-01-26T09:54:14.196800+00:00",
    "intent": "setup",
    "notes": []
  }
}
Request status:
<Response [201]>
{
  "data": {
    "id": "a91f7f83-c887-41b1-884d-958def56f682",
    "createdAt": "2025-01-26T09:54:14.455160+00:0

In [79]:
frame.shape

(1944, 2592, 3)

In [58]:

for k in range(5):
    i = 0
    while i < 5:
        openapi.move_to_coordinates((X + i*3, Y - k*3, dish_bottom+pickup_offset), min_z_height=dish_bottom)
        time.sleep(1)
        r = openapi.aspirate_in_place(flow_rate = flow_rate, volume = 50, verbose=True)
        responce_dict = json.loads(r.text)['data']
        if responce_dict['status'] == 'failed':
            if responce_dict['error']['errorType'] == 'InvalidAspirateVolumeError':
                print('Dumping fluid')
                openapi.move_relative('z', 50)
                openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
                openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
                openapi.dispense_in_place(flow_rate = 200, volume = 10)
                continue
        else:
            for j in range(6):
                openapi.move_relative('z', 0.5)
                time.sleep(0.3)
            i += 1
    openapi.move_relative('z', 50)
    openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
    openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
    openapi.dispense_in_place(flow_rate = 200, volume = 10)  
            
    

Request status:
<Response [201]>
{
  "data": {
    "id": "1b728c34-3512-498e-a81e-5ece96bab552",
    "createdAt": "2025-01-25T16:53:38.806800+00:00",
    "commandType": "moveToCoordinates",
    "key": "1b728c34-3512-498e-a81e-5ece96bab552",
    "status": "succeeded",
    "params": {
      "minimumZHeight": 9.4,
      "forceDirect": false,
      "pipetteId": "cd4bc248-9f01-46a9-bf8e-c2905e3b7fee",
      "coordinates": {
        "x": 185.9910805734899,
        "y": 132.9491814244559,
        "z": 9.4
      }
    },
    "result": {
      "position": {
        "x": 185.9910805734899,
        "y": 132.9491814244559,
        "z": 9.4
      }
    },
    "startedAt": "2025-01-25T16:53:38.808968+00:00",
    "completedAt": "2025-01-25T16:53:40.414702+00:00",
    "intent": "setup",
    "notes": []
  }
}
Request status:
<Response [201]>
{
  "data": {
    "id": "af68ee0d-3e02-4835-be6c-737e6a0dc986",
    "createdAt": "2025-01-25T16:53:41.457086+00:00",
    "commandType": "aspirateInPlace",
    "key

In [56]:
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
openapi.dispense_in_place(flow_rate = 200, volume = 10)

<Response [201]>

In [44]:
openapi.move_relative('z', 50)
openapi.blow_out(openapi.labware_dct['3'], "A1", well_location='center', flow_rate = 200)
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 10, flow_rate = 200)
openapi.dispense_in_place(flow_rate = 200, volume = 10)

<Response [201]>

In [30]:
openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='center', volume = 100, flow_rate = 200)

<Response [201]>

In [31]:
openapi.dispense(openapi.labware_dct['3'], "A1", well_location='center', volume = 100, flow_rate = 200)

<Response [201]>

# Hydrogel core detection

In [63]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])

def on_mouse_click(event, x, y, flags, param):
    global circle_center, circle_radius

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

# Create an instance of the ManualRobotMovement class
manual_movement = utils.ManualRobotMovement(openapi)

window = cap.get_window()
cv2.setMouseCallback(cap.window_name, on_mouse_click)

target_x, target_y = 0, 0

dish_bottom = 9.4# - 11.5
pickup_offset = 0 #0.6
flow_rate = 300
volume = 50

while True:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 190), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Flow rate: {flow_rate} uL/s", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Volume: {volume} uL", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Pickup height: {dish_bottom + pickup_offset} mm", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # masked_frame = cv2.bitwise_and(frame, mask)

    center_screen_x = frame.shape[1] // 2
    center_screen_y = frame.shape[0] // 2
    cv2.circle(frame, (target_x, target_y), 3, (0, 0, 255), -1)

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(gray_img, dtype=np.uint8)
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)
    gray_img = cv2.GaussianBlur(gray_img, (21, 21), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_img)
    thresh_img = cv2.adaptiveThreshold(clahe_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 65, 9)
    kernel = np.ones((7, 7), np.uint8)
    opening = cv2.morphologyEx(thresh_img, cv2.MORPH_OPEN, kernel)
    masked_opening = cv2.bitwise_and(opening, mask)
    contours, _ = cv2.findContours(masked_opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_contour_area = 500
    max_contour_area = 5000

    circular_contours = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        area = cv2.contourArea(contour)
        if perimeter == 0 or area < min_contour_area or area > max_contour_area:
            continue
        circularity = 4 * np.pi * (area / (perimeter * perimeter))
        if 0.7 < circularity < 1.3:  # Adjust the range as needed
            circular_contours.append(contour)

    contours = circular_contours

    contour_centers = []
    for contour in contours:
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            contour_centers.append((cX, cY))
        else:
            contour_centers.append((0, 0))

    contour_img = frame.copy()
    cv2.circle(contour_img, circle_center, circle_radius, (255, 0, 0), 2)
    cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
    # Draw contour centers on the image
    for center in contour_centers:
        cv2.circle(contour_img, center, 5, (255, 0, 0), -1)
    
    cv2.imshow(cap.window_name, contour_img)
    # Draw a point at the center of the screen
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

    elif key_pressed == ord('s'):
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H-%M-%S')
        filename = f"frame_{timestamp}.png"
        cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+filename, frame)
        print(f"Frame saved as {filename}")


cv2.destroyAllWindows()

In [64]:
converted_contour_centers = []
x, y, _ = openapi.get_position(verbose=False)[0].values()
for center in contour_centers:
    cX, cY = center
    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)
    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
    X = X_init + diff[0] + offset[0]
    Y = Y_init + diff[1] + offset[1]
    converted_contour_centers.append((X, Y))

In [65]:
len(converted_contour_centers)

25

In [42]:
for i in range(10):
    x, y = converted_contour_centers[i]
    openapi.move_to_coordinates((x,y,12), min_z_height=1, verbose=False)
    time.sleep(0.1)

openapi.move_relative('z', 50)
    

# Picking procedure (semi-automatic)

In [14]:
cr = core.Core()

In [110]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])
size_conversion_ratio = calibration_data['size_conversion_ratio']
one_d_ratio = calibration_data['one_d_ratio']

vol = 10
dish_bottom = 9.5 #10.60 for 300ul, 9.5 for 200ul
pickup_offset = 0.5
pickup_height = dish_bottom + pickup_offset
flow_rate = 50
cuboid_size_theshold = (350, 450)
failure_threshold = 0.5
minimum_distance = 2

window = cap.get_window()

def create_well_mapping():
    rows = list("ABCDEFGHIJKLMNOP")
    columns = list(range(1, 25))
    well_mapping = {}

    for i in range(384):
        row = rows[i // 24]
        column = columns[i % 24]
        well_mapping[i] = f"{row}{column}"

    return well_mapping

well_mapping = create_well_mapping()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    if event == cv2.EVENT_LBUTTONDBLCLK:
        for contour in filtered_contours:
            r=cv2.pointPolygonTest(contour, (x,y), False)
            if r>0:
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

                    x, y, _ = openapi.get_position(verbose=False)[0].values()
                    diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
                    X = X_init + diff[0] + offset[0]
                    Y = Y_init + diff[1] + offset[1]
                    
                    print(f"Robot coords: ({x}, {y})")
                    print(f"Clicked on: ({X}, {Y})")
                    openapi.move_to_coordinates((X, Y, 20), min_z_height=1, verbose=False)
                    time.sleep(0.1)
                    openapi.aspirate_in_place(flow_rate = flow_rate, volume = vol)
                    time.sleep(0.1)
                    openapi.move_to_coordinates((X, Y, pickup_height), min_z_height=1, verbose=False, force_direct=True)
                    time.sleep(1)
                    openapi.move_to_coordinates((X, Y, 30), min_z_height=1, verbose=False)
                else:
                    print("Contour center could not be found")

    if event == cv2.EVENT_RBUTTONDOWN:
        x, y, _ = openapi.get_position(verbose=False)[0].values()
        # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 700
manual_movement = utils.ManualRobotMovement(openapi)

# Which wells to fill
columns = list(range(1,25))
#rows = ['F', 'G', 'H', 'I']
rows = ['G']

wells_to_fill = [f"{row}{column}" for row in rows for column in columns]

reversed_well_mapping = {v: k for k, v in well_mapping.items()}
wells_to_fill_indices = [reversed_well_mapping[well] for well in wells_to_fill]

idx = wells_to_fill_indices[0]
end_idx = wells_to_fill_indices[-1] + 1

cuboid_chosen = False
cuboid_choice = None
while idx < end_idx:
    frame = cap.get_frame(undist=True)
    x, y, z = openapi.get_position(verbose=False)[0].values()
    (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.circle(frame, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (0, 0, 255), 2)
    mask = np.zeros_like(frame, dtype=np.uint8)
    cv2.circle(mask, circle_center, circle_radius + int(minimum_distance / one_d_ratio), (255, 255, 255), -1)
    masked_frame = cv2.bitwise_and(frame, mask)
    cv2.circle(frame, circle_center, circle_radius, (0, 255, 0), 2)

    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,2) 
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)

    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cr.cuboids = contours
    filtered_contours = [contour for contour in cr.cuboids if 30 < cv2.contourArea(contour) < 300]
    cr.cuboid_dataframe(filtered_contours)

    cuboid_size_micron2 = cr.cuboid_df.area * size_conversion_ratio * 1000000
    cuboid_diameter = 2 * np.sqrt(cuboid_size_micron2 / np.pi)
    dist_mm = cr.cuboid_df.min_dist * one_d_ratio
    cr.cuboid_df['diameter_microns'] = cuboid_diameter
    cr.cuboid_df['min_dist_mm'] = dist_mm
    
    # Filter out elongated contours

    pickable_cuboids = cr.cuboid_df.loc[(cuboid_size_theshold[0] < cr.cuboid_df['diameter_microns']) & 
                                        (cr.cuboid_df['diameter_microns'] < cuboid_size_theshold[1]) &
                                        ((cr.cuboid_df['aspect_ratio'] > 0.75) | (cr.cuboid_df['aspect_ratio'] < 1.25)) &
                                        (cr.cuboid_df['circularity'] > 0.6)].copy()

    # Check if cuboid centers are within the circle radius from the current circle center
    pickable_cuboids['distance_to_center'] = pickable_cuboids.apply(
        lambda row: np.sqrt((row['cX'] - circle_center[0])**2 + (row['cY'] - circle_center[1])**2), axis=1
    )
    pickable_cuboids = pickable_cuboids[pickable_cuboids['distance_to_center'] <= circle_radius]
    isolated = pickable_cuboids.loc[pickable_cuboids.min_dist_mm > minimum_distance]
    draw = isolated.contour.values.tolist()
    cv2.drawContours(frame, filtered_contours, -1, (0, 0, 255), 2)
    cv2.drawContours(frame, pickable_cuboids.contour.values.tolist(), -1, (0, 255, 255), 2)
    cv2.drawContours(frame, draw, -1, (0, 255, 0), 2)

    if not cuboid_chosen and len(isolated) > 0:
        if cuboid_choice is not None:
            prev_x, prev_y = cuboid_choice[['cX', 'cY']].values[0]
            
            cv2.circle(frame, (int(prev_x), int(prev_y)), int(round(failure_threshold / one_d_ratio)), (255, 0, 0), 2)
            distances = cr.cuboid_df.apply(lambda row: np.sqrt((row['cX'] - prev_x)**2 + (row['cY'] - prev_y)**2), axis=1).to_numpy()
            distances *= one_d_ratio
            if any(distances <= failure_threshold):
                print("Miss detected ...")
                idx -= 1

        cuboid_choice = isolated.sample(n=1)
        cuboid_chosen = True
        cv2.imwrite(str(paths.BASE_DIR)+'\\outputs\\images\\'+f"frame_{idx}.png", frame)
        

    # for i, row in cr.cuboid_df.iterrows():
    #     cX, cY = int(row['cX']), int(row['cY'])
    #     aspect_ratio = row['aspect_ratio']
    #     circularity = row['circularity']
    #     cv2.putText(frame, f"{aspect_ratio:.2f}, {circularity:.2f}", (cX+20, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    if cuboid_chosen:    
        cv2.drawContours(frame, cuboid_choice.contour.values.tolist(), -1, (255, 0, 0), 2)

    cv2.imshow(cap.window_name, frame)

    #-----------------------------------------------INTERACTION------------------------------------------------
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        break

    elif key_pressed == ord('c'):
        if len(isolated) > 0:
            cuboid_choice = isolated.sample(n=1)

    elif key_pressed == ord('p'):
        if len(isolated) == 0:
            print("No cuboids found in the selected region")
            continue

        cX, cY = cuboid_choice[['cX', 'cY']].values[0]

        X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

        x, y, _ = openapi.get_position(verbose=False)[0].values()
        diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
        X = X_init + diff[0] + offset[0]
        Y = Y_init + diff[1] + offset[1]
        
        openapi.move_to_coordinates((X, Y, pickup_height), min_z_height=dish_bottom, verbose=False)
        openapi.aspirate_in_place(flow_rate = flow_rate, volume = vol)
        # ---------------------------------WHERE TO DISPENSE---------------------------------------
        # x_d, y_d = converted_contour_centers[idx]
        # openapi.move_to_coordinates((x_d,y_d, dish_bottom + 2.5), min_z_height=1, verbose=False)
        # time.sleep(0.5)
        # openapi.dispense_in_place(flow_rate = flow_rate, volume = vol)
        # time.sleep(0.5)
        openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', volume = vol, flow_rate = flow_rate)
        time.sleep(0.3)
        openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=dish_bottom, verbose=False)
        idx += 1
        cuboid_chosen = False
        time.sleep(0.1)

    elif key_pressed == ord('d'):
        r = openapi.dispense_in_place(flow_rate = flow_rate, volume = vol)


cv2.destroyAllWindows()
keyboard.unhook_all()

No cuboids found in the selected region
No cuboids found in the selected region
No cuboids found in the selected region
No cuboids found in the selected region
No cuboids found in the selected region


In [10]:
openapi.blow_out_in_place(50)

<Response [201]>

In [105]:
r = openapi.dispense(openapi.labware_dct['3'], "A1", well_location='top', volume = 10, flow_rate = 5, pushout=0)

In [11]:
r = openapi.aspirate(openapi.labware_dct['3'], "A1", well_location='top', offset = (0,0,3), volume = 10, flow_rate = 5)

In [ ]:
r = openapi.dispense(openapi.labware_dct['6'], well_mapping[idx], well_location='bottom', volume = vol, flow_rate = flow_rate)

In [88]:
json.loads(r.text)

{'data': {'id': 'c880966c-43ac-43d9-b0d4-2ee34b3a81bb',
  'createdAt': '2025-01-26T12:10:58.855084+00:00',
  'commandType': 'dispense',
  'key': 'c880966c-43ac-43d9-b0d4-2ee34b3a81bb',
  'status': 'failed',
  'params': {'labwareId': '492ea99c-5b92-4999-816b-6cda6d1c1d6f',
   'wellName': 'A1',
   'wellLocation': {'origin': 'bottom',
    'offset': {'x': 0, 'y': 0, 'z': 0},
    'volumeOffset': 0.0},
   'flowRate': 100.0,
   'volume': 1.0,
   'pipetteId': '6a51cb8a-aa10-45a7-bb93-666fb4664a70',
   'pushOut': 30.0},
  'error': {'id': '75a499ed-c890-4f81-a634-67511d459c77',
   'createdAt': '2025-01-26T12:10:58.892388+00:00',
   'isDefined': False,
   'errorType': 'CommandPreconditionViolated',
   'errorCode': '4004',
   'detail': 'Cannot push_out on a dispense that does not leave the pipette empty',
   'errorInfo': {'command': 'dispense', 'remaining-volume': '9.0'},
   'wrappedErrors': []},
  'startedAt': '2025-01-26T12:10:58.857117+00:00',
  'completedAt': '2025-01-26T12:10:58.892388+00:00'

# Cuboid recognition

In [ ]:
calibration_profile = 'standardDeck'
calibration_data = utils.load_calibration_config(calibration_profile)

tf_mtx = np.array(calibration_data['tf_mtx'])
calib_origin = np.array(calibration_data['calib_origin'])[:2]
offset = np.array(calibration_data['offset'])


window = cap.get_window()

def on_mouse_click(event, x, y, flags, param):
    global circle_center
    global circle_radius
    global filtered_contours
    global X_init, Y_init

    if event == cv2.EVENT_MBUTTONDOWN:
        circle_center = (x, y)

    if event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:
            circle_radius += 10
        else:
            circle_radius -= 10

    # if event == cv2.EVENT_LBUTTONDBLCLK:
    #     for contour in filtered_contours:
    #         r=cv2.pointPolygonTest(contour, (x,y), False)
    #         if r>0:
    #             M = cv2.moments(contour)
    #             if M["m00"] != 0:
    #                 cX = int(M["m10"] / M["m00"])
    #                 cY = int(M["m01"] / M["m00"])
    #                 X_init, Y_init, _ = tf_mtx @ (cX, cY, 1)

    #                 x, y, _ = openapi.get_position(verbose=False)[0].values()
    #                 diff = np.array([x,y]) - np.array(calibration_data['calib_origin'])[:2]
    #                 X = X_init + diff[0] + offset[0]
    #                 Y = Y_init + diff[1] + offset[1]
                    
    #                 print(f"Robot coords: ({x}, {y})")
    #                 print(f"Clicked on: ({X}, {Y})")
    #                 openapi.move_to_coordinates((X, Y, 15), min_z_height=1, verbose=False)
    #                 # openapi.aspirate_in_place(flow_rate = 75, volume = 10)

                    
    #             else:
    #                 print("Contour center could not be found")

    # if event == cv2.EVENT_RBUTTONDOWN:
    #     x, y, _ = openapi.get_position(verbose=False)[0].values()
    #     # openapi.move_to_coordinates((x, y, 100), min_z_height=1)
    #     openapi.move_to_coordinates((calib_origin[0],calib_origin[1],100), min_z_height=1, verbose=False)

        

cv2.setMouseCallback(cap.window_name, on_mouse_click)
circle_center = (int(1296.0), int(972.0))
circle_radius = 300
# manual_movement = utils.ManualRobotMovement(openapi)

while True:
    frame = cap.get_frame(undist=True)
    # x, y, z = openapi.get_position(verbose=False)[0].values()
    # (text_width, text_height), _ = cv2.getTextSize(f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    # cv2.rectangle(frame, (10, 0), (10 + text_width, text_height + 70), (0, 0, 0), -1)
    # cv2.putText(frame, f"Robot coords: ({x:.2f}, {y:.2f}, {z:.2f})", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # cv2.putText(frame, f"Step size: {manual_movement.step} mm", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.circle(frame, circle_center, circle_radius, (255, 0, 0), 2)
    # Create a mask with the same dimensions as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Draw a filled circle on the mask
    cv2.circle(mask, circle_center, circle_radius, (255, 255, 255), -1)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convert the masked frame to grayscale
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to the grayscale image
    # _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)

    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,25,2) 
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    # Fill the area outside the circle with black pixels
    # Convert the mask to grayscale
    mask_inv = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    thresh = cv2.bitwise_and(thresh, mask_inv)

    # frame[thresh == 255] = [0, 255, 0]

    # Find contours in the masked frame
    contours, hei = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)



    # Filter the contours to exclude the outermost
    filtered_contours = [contour for contour, h in zip(contours, hei[0]) if h[3] == 1]
    # Filter the contours by size
    filtered_contours = [contour for contour in contours if 15 < cv2.contourArea(contour) < 500]
    # Draw the contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)

    cv2.imshow(cap.window_name, frame)
    key_pressed = cv2.waitKey(1)

    if key_pressed == ord('q'):
        keyboard.unhook_all()
        break

        
cv2.destroyAllWindows()

In [33]:
json.loads(r.text)

{'data': {'id': '9aebde82-ea2c-4b57-84a0-49c354c553b1',
  'createdAt': '2024-11-02T23:50:05.116797+00:00',
  'commandType': 'dispenseInPlace',
  'key': '9aebde82-ea2c-4b57-84a0-49c354c553b1',
  'status': 'failed',
  'params': {'flowRate': 10.0,
   'volume': 10.0,
   'pipetteId': '503cdce5-efa2-4cf0-bf63-8f60653d1b34',
   'pushOut': 10.0},
  'error': {'id': '97cac116-5267-4330-b984-7ee9673985ca',
   'createdAt': '2024-11-02T23:50:05.123979+00:00',
   'isDefined': False,
   'errorType': 'InvalidDispenseVolumeError',
   'errorCode': '4000',
   'detail': 'Cannot dispense 10.0 µL when only 0.0 µL has been aspirated.',
   'errorInfo': {},
   'wrappedErrors': []},
  'startedAt': '2024-11-02T23:50:05.118798+00:00',
  'completedAt': '2024-11-02T23:50:05.123979+00:00',
  'intent': 'setup',
  'notes': []}}

In [29]:
openapi.dispense_in_place(flow_rate = 100, volume = 10)

<Response [201]>

In [37]:
cuboid_choice = isolated.sample(n=1)

In [39]:
cX, cY = cuboid_choice[['cX', 'cY']].values[0]

In [16]:
cr.cuboid_df.apply(lambda row : cr.contour_centers([row.values[0]])[0], axis=1)

0      (1312, 1256)
1      (1282, 1254)
2      (1298, 1253)
3      (1209, 1242)
4      (1303, 1235)
           ...     
196     (1186, 718)
197     (1269, 713)
198     (1389, 710)
199     (1261, 710)
200     (1359, 696)
Length: 201, dtype: object